<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/note_books/lgbm_26_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import shutil
import time
import sys
import math
from contextlib import contextmanager

In [5]:
N_SPLITS = 10
SEED = 42


In [6]:
LOG_PATH = Path("./log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)


In [4]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


In [18]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, x, y):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2))
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df

In [8]:
set_seed(SEED)


In [32]:
feature_dir = "/content/drive/MyDrive/wifi_with_floor"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train_with_floor.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test_with_floor.csv')))
subm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv', index_col=0)

In [22]:
a = pd.read_csv('/content/drive/MyDrive/wifi_with_floor/5a0546857ecc773753327266_test_with_floor.csv')
a

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd5da29ea31fc8fa2315f92f0c067e4e24e42b05,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,floor,site_path_timestamp,path,timestamp
0,0,-51,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-71,-55,-999,-75,-999,-999,-999,-999,-66,-47,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-83,-999,-999,-83,-999,-78,-999,-63,-999,-999,-61,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,0,5a0546857ecc773753327266_046cfa46be49fc1083481...,046cfa46be49fc10834815c6,9
1,1,-55,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-65,-999,-82,-999,-999,-999,-999,-66,-65,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-77,-999,-999,-86,-999,-75,-999,-70,-999,-999,-57,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-89,-999,-999,-999,0,5a0546857ecc773753327266_046cfa46be49fc1083481.

In [19]:
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

In [41]:
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y = np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        
        modelx = lgb.LGBMRegressor(**lgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

            
        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)

        preds_x += modelx.predict(test_data.iloc[:, :-3]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-3]) / N_SPLITS
        preds_f = test_data['floor'].values

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], y_validx.to_numpy(), y_validy.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)
    
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

[5.5GB(+0.0GB): 16.279sec] fit X
[5.5GB(+0.0GB): 15.259sec] fit Y


fold 0: mean position error 3.2717038664893394


[5.5GB(+0.0GB): 16.845sec] fit X
[5.5GB(+0.0GB): 11.680sec] fit Y


fold 1: mean position error 3.3365560433976653


[5.5GB(+0.0GB): 12.857sec] fit X
[5.5GB(+0.0GB): 20.344sec] fit Y


...

fold 7: mean position error 4.144860688081646


[11.7GB(+0.0GB): 54.796sec] fit X
[11.7GB(+0.0GB): 40.596sec] fit Y


fold 8: mean position error 4.05567004793882


[11.7GB(+0.0GB): 44.525sec] fit X
[11.7GB(+0.0GB): 49.448sec] fit Y


fold 9: mean position error 4.231137843350287
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #23, shape=(15655, 4869), name=5dc8cea7659e181adb076a3f_train_with_floor.csv
mean position error 4.132929702969397
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [42]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

In [43]:
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,92.715723,99.588380
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,91.450002,103.519300
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,88.727696,105.501512
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,89.482356,106.187738
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,91.422920,109.680115
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,5,213.447498,94.842348
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,5,215.058525,98.456227
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,5,206.135842,107.473748


In [44]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission_lgbm_26.csv')

In [45]:
!mv /content/submission_lgbm_26.csv /content/drive/MyDrive

In [46]:
floor = pd.read_csv('/content/drive/MyDrive/only_accurate_floor.csv')

In [47]:
all_preds['floor'] = floor['floor'].values

In [48]:
all_preds.to_csv('submission_lgbm_27.csv')

In [49]:
!mv /content/submission_lgbm_27.csv /content/drive/MyDrive

In [ ]:
pd.